<a href="https://colab.research.google.com/github/wfos3241/TextMining/blob/main/ex01%EC%98%81%EB%AC%B8%EC%98%81%ED%99%94%EB%A6%AC%EB%B7%B0%EB%8D%B0%EC%9D%B4%ED%84%B0%EB%B6%84%EB%A5%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 데이터셋
- 영문으로 된 영화리뷰 데이터 셋
- 라벨 : neg(qnwjd), pos(긍정), unsup(라벨이 없음을 의미 - 비지도학습용)
- 50000개 데이터로 구성


In [1]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd /content/drive/MyDrive/Colab Notebooks/텍스트마이닝

/content/drive/MyDrive/Colab Notebooks/텍스트마이닝


In [3]:
# 현재 작업 폴더를 출력

!pwd

/content/drive/MyDrive/Colab Notebooks/텍스트마이닝


- 데이터 파일 가져오기

In [4]:
import pandas as pd
imdb = pd.read_csv("./data/imdb_master.csv", encoding = "latin-1")
imdb.head()

,Unnamed: 0,type,review,label,file
0,0,test,Once again Mr. Costner has dragged out a movie...,neg,0_2.txt
1,1,test,This is an example of why the majority of acti...,neg,10000_4.txt
2,2,test,"First of all I hate those moronic rappers, who...",neg,10001_1.txt
3,3,test,Not even the Beatles could write songs everyon...,neg,10002_3.txt
4,4,test,Brass pictures (movies is not a fitting word f...,neg,10003_3.txt


- review, label만 추출

In [5]:
imdb2 = imdb.loc[:, "review":"label"]
imdb2.head()

,review,label
0,Once again Mr. Costner has dragged out a movie...,neg
1,This is an example of why the majority of acti...,neg
2,"First of all I hate those moronic rappers, who...",neg
3,Not even the Beatles could write songs everyon...,neg
4,Brass pictures (movies is not a fitting word f...,neg


- EDA (탐색적 데이터 분석)
  - 데이터를 수정하기 않고 눈으로 확인하는 작업
      - 오류, 결측치, 이상치 유무 확인
      - 편향 유무
      - 분산, 평균 등 확인
      - 시각화

- 특성공학 (Feature Engineering)
  - 요구사항에 맞게 데이터를 가공하는 작업


- 라벨의 클래스별 갯수를 확인 -> 범주형 데이터의 편향을 확인
  -> 한쪽으로 데이터의 수가 치우쳐 있으면 많은 수를 갖는 클래스에 편향되어서 예측

In [6]:
imdb2["label"].value_counts()

,count
label,
unsup,50000
neg,25000
pos,25000


- label 컬럼에서 unsup 클래스를 삭제

In [7]:
# label 컬럼의 클래스가 unsup이 아닌 것만 저장

imdb3 = imdb2[imdb2["label"] != "unsup"]
imdb3["label"].value_counts()

,count
label,
neg,25000
pos,25000


- clean(정제) : review에서 영문, 빈 공백만 추출

In [8]:
import re

# "^a-zA-Z" : 영문 소문자, 대문자, 빈공백이 아닌 것
# replace("[^a-zA-Z ]", "") : 영문 소문자, 대문자, 빈공백이 아닌 것만 삭제

imdb3["review"] = imdb3["review"].str.replace("[^a-zA-Z ]", "")
imdb3.head()

<ipython-input-8-9dd57b55b630>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  imdb3["review"] = imdb3["review"].str.replace("[^a-zA-Z ]", "")


,review,label
0,Once again Mr. Costner has dragged out a movie...,neg
1,This is an example of why the majority of acti...,neg
2,"First of all I hate those moronic rappers, who...",neg
3,Not even the Beatles could write songs everyon...,neg
4,Brass pictures (movies is not a fitting word f...,neg


- 텍스트에서 영문 또는 한글만 추출하게 되면 빈 텍스트가 발생할 수 있음

In [9]:
# review 컬럼에 빈 데이터가 있는지 확인

imdb3 = imdb3[imdb3["review"] != ""]
imdb3.head()

,review,label
0,Once again Mr. Costner has dragged out a movie...,neg
1,This is an example of why the majority of acti...,neg
2,"First of all I hate those moronic rappers, who...",neg
3,Not even the Beatles could write songs everyon...,neg
4,Brass pictures (movies is not a fitting word f...,neg


- 데이터 갯수 축소

In [10]:
# imdb3 = imdb3.sample(n = 5000)

- 훈련/검증 데이터로 분리

In [11]:
import numpy as np
from sklearn.model_selection import train_test_split

X = imdb3["review"]
y = imdb3["label"]

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.25, random_state = 42)
X_train.shape, X_val.shape, y_train.shape, y_val.shape

((37500,), (12500,), (37500,), (12500,))

In [12]:
print(y_train.value_counts())
print(y_val.value_counts())

label
pos    18790
neg    18710
Name: count, dtype: int64
label
neg    6290
pos    6210
Name: count, dtype: int64


- 토큰화/인코딩
  - CountVectotizer()
  - TfidVectotizer()

In [31]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

cv = CountVectorizer(max_features = 10000)
cv.fit(X_train)
tv = TfidfVectorizer(max_features = 10000)
tv.fit(X_train)

TfidfVectorizer(max_features=10000)

In [32]:
X_train_cv = cv.transform(X_train).toarray()
X_val_cv = cv.transform(X_val).toarray()
X_train_tv = tv.transform(X_train).toarray()
X_val_tv = tv.transform(X_val).toarray()

- 학습

In [33]:
from sklearn.linear_model import LogisticRegression

lr_cv = LogisticRegression(C = 0.1, max_iter = 15000)
lr_cv.fit(X_train_cv, y_train)

LogisticRegression(C=0.1, max_iter=15000)

In [34]:
print("훈련 정확도 : ", lr_cv.score(X_train_cv, y_train))
print("검증 정확도 : ", lr_cv.score(X_val_cv, y_val))

훈련 정확도 :  0.95208
검증 정확도 :  0.8932


In [48]:
from sklearn.linear_model import LogisticRegression

lr_tv = LogisticRegression(C = 10, max_iter = 15000)
lr_tv.fit(X_train_tv, y_train)

LogisticRegression(C=10, max_iter=15000)

In [49]:
print("훈련 정확도 : ", lr_tv.score(X_train_tv, y_train))
print("검증 정확도 : ", lr_tv.score(X_val_tv, y_val))

훈련 정확도 :  0.9556533333333334
검증 정확도 :  0.89496


In [50]:
pred_cv = lr_cv.predict(X_val_cv)

for i in range(10):
  print(pred_cv[i], " ==> ", y_val.iloc[i])

neg  ==>  neg
neg  ==>  neg
neg  ==>  neg
neg  ==>  neg
pos  ==>  pos
pos  ==>  pos
neg  ==>  neg
pos  ==>  pos
neg  ==>  neg
neg  ==>  neg


In [51]:
pred_tv = lr_tv.predict(X_val_tv)

for i in range(10):
  print(pred_tv[i], " ==> ", y_val.iloc[i])

neg  ==>  neg
neg  ==>  neg
neg  ==>  neg
neg  ==>  neg
pos  ==>  pos
pos  ==>  pos
neg  ==>  neg
pos  ==>  pos
neg  ==>  neg
neg  ==>  neg


# 임베딩한 결과로 학습 - FastText

In [57]:
!pip install gensim

In [58]:
import nltk

nltk.download("punkt_tab")

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [36]:
from gensim.models import FastText
from nltk.tokenize import word_tokenize

def tokenize_texts(texts) :
  return [word_tokenize(text) for text in texts]

X_train_token = tokenize_texts(X_train)
X_val_token = tokenize_texts(X_val)

fasttext_model = FastText(sentences = X_train_token, vector_size = 100, window = 3, min_count = 1, sg = 1)

- 문장의 벡터로 변환 (임베딩) -> word embedding
  - 희소벡터 (sparse vector) : 대부분 0인 데이터 -> CountVectorizer(정수), TfidfVectorizer(실수)
  - 밀집벡터 (dense vector) : 대부분의 데이터가 값을 갖는 데이터 -> 임베딩(실수)

In [40]:
def get_sentence_vector(tokens, model):

  vectors = [model.wv[word] for word in tokens if word in model.wv]
  return np.mean(vectors, axis=0) if vectors else np.zeros(model.vector_size)

X_train_vec = [get_sentence_vector(tokens, fasttext_model) for tokens in X_train_token]
X_val_vec = [get_sentence_vector(tokens, fasttext_model) for tokens in X_val_token]

In [38]:
from sklearn.linear_model import LogisticRegression

lr_fasttext = LogisticRegression(C = 10, max_iter = 15000)
lr_fasttext.fit(X_train_vec, y_train)

LogisticRegression(C=10, max_iter=15000)

In [41]:
print("훈련 정확도 : ", lr_fasttext.score(X_train_vec, y_train))
print("검증 정확도 : ", lr_fasttext.score(X_val_vec, y_val))

훈련 정확도 :  0.85184
검증 정확도 :  0.84672


# RandomForest 모델로 학습하기

In [42]:
from sklearn.ensemble import RandomForestClassifier

rfc_fasttext = RandomForestClassifier(random_state = 10, n_estimators = 100)
rfc_fasttext.fit(X_train_vec, y_train)

RandomForestClassifier(random_state=10)

In [43]:
print("훈련 점수 : ", rfc_fasttext.score(X_train_vec, y_train))
print("테스트 점수 : ", rfc_fasttext.score(X_val_vec, y_val))

훈련 점수 :  1.0
테스트 점수 :  0.80328
